In [30]:
import d2l
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn, utils
import numpy as np
import pandas as pd
import gluonnlp as nlp
from utils.preprocessing import one_hot_encoder
from utils.preprocessing import missing_values_table


In [2]:
spec_feature = pd.read_csv('/Applications/files/classes_homework/Berkeley_ieor/data-x/project/feature_matrix_spec.csv')

In [3]:
missing_table = missing_values_table(spec_feature)

dump_feats = missing_table[missing_table['% of Total Values'] > 30].index.tolist()

spec_feature = spec_feature.drop(dump_feats, axis = 1)

spec_feature, _ = one_hot_encoder(spec_feature)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer

Target = spec_feature['TARGET']
ID = spec_feature['SK_ID_CURR']

dataset_temp = spec_feature.drop(['TARGET','SK_ID_CURR'], axis = 1)

imputer = Imputer(strategy = 'median')
scaler = MinMaxScaler(feature_range = (0,1))

dataset_preprocessed = imputer.fit_transform(dataset_temp)
dataset_preprocessed = scaler.fit_transform(dataset_preprocessed)
dataset_preprocessed = pd.DataFrame(dataset_preprocessed, columns = dataset_temp.columns)

dataset_preprocessed['TARGET'] = Target
dataset_preprocessed['SK_ID_CURR'] = ID

There are total 885 columns.
823 of them have missing values.


In [4]:
dataset_preprocessed = dataset_preprocessed[dataset_preprocessed.TARGET != -999]

In [5]:
X_ = dataset_preprocessed.drop(['TARGET','SK_ID_CURR'], axis=1)
y_ = dataset_preprocessed['TARGET']
#print(X_.shape, y_.shape)


In [6]:
from sklearn import model_selection
train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X_, y_, test_size=0.3, random_state=42)

In [7]:
print(train_features.shape, train_labels.shape)

print(test_features.shape, test_labels.shape)

(215257, 789) (215257,)
(92254, 789) (92254,)


In [8]:
train_features = nd.array(train_features)
train_labels = nd.array(train_labels)
test_features = nd.array(test_features)
test_labels = nd.array(test_labels)

In [9]:
train_features.shape

(215257, 789)

### Build Neural Network

In [96]:
def get_net(drop_prob1,drop_prob2):
    net = nn.Sequential()
    net.add(nn.Dense(789, activation="relu"),
            # Add a dropout layer after the first fully connected layer
            nn.Dropout(drop_prob1),
            nn.Dense(128),
            # Add a dropout layer after the second fully connected layer
            nn.Dropout(drop_prob2),
            nn.Dense(2)
           )
    net.initialize(init.Normal(sigma=0.01))
    return net
net = get_net(0.1,0)
    

### Train
 Use cross entropy as loss

In [97]:

batch_size = 128
loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01, 'wd':0.1})
num_epochs = 10


train_iter = gdata.DataLoader(gdata.ArrayDataset(
        train_features, train_labels), batch_size, shuffle=True)
test_iter = gdata.DataLoader(gdata.ArrayDataset(
        test_features, test_labels), batch_size, shuffle=True)

d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
              None, trainer)

epoch 1, loss 0.3028, train acc 0.919, test acc 0.920
epoch 2, loss 0.2820, train acc 0.919, test acc 0.920
epoch 3, loss 0.2817, train acc 0.919, test acc 0.920
epoch 4, loss 0.2817, train acc 0.919, test acc 0.920
epoch 5, loss 0.2816, train acc 0.919, test acc 0.920
epoch 6, loss 0.2816, train acc 0.919, test acc 0.920
epoch 7, loss 0.2816, train acc 0.919, test acc 0.920
epoch 8, loss 0.2816, train acc 0.919, test acc 0.920
epoch 9, loss 0.2816, train acc 0.919, test acc 0.920
epoch 10, loss 0.2815, train acc 0.919, test acc 0.920


In [99]:
from sklearn import metrics
preds = net(test_features).asnumpy()
nn_auc = metrics.roc_auc_score(test_labels.asnumpy(),preds[:,1])
print('AUC:',nn_auc)

AUC: 0.5374381970741873
